In [1]:
import pandas as pd

from ast import literal_eval #to convert list red as string back to list

In [40]:
data = pd.read_csv('dataset.csv')
data = data.loc[:,['job_title', 'job_company', 'job_details']]
data.head()

,job_title,job_company,job_details
0,IT Data Engineer / Data Analytics,PT Kawan Lama Sejahtera,"['Jakarta', 'IDR\xa010.000.000 - 20.000.000', ..."
1,Data Engineer,Mapan - PT Ruma (Rekan Usaha Mikro Anda),"['SCBD, Jakarta', '1 – 3 tahun']"
2,Business Analyst (Afees),AIA Singapore Pte Ltd,"['Batam', '3 – 5 tahun']"
3,e-Commerce Specialist (Remote),Etaily,"['Jakarta', 'Login to view estimated salary by..."
4,Digital Marketing Manager (Remote),Hepelle,"['Jakarta', 'Login to view estimated salary by..."


# DATA CLEANING

In [3]:
data['job_details'] = data['job_details'].map(lambda x: literal_eval(x)) #convert list red as string back to list

In [4]:
#splitting job_details
data['location'] = data['job_details'].map(lambda x: x[0])
data['experience'] = data['job_details'].map(lambda x: x[-1])
data['salary'] = data['job_details'].map(lambda x: x[1] if len(x)==3 else '')

In [5]:
#Cleaning location to specific City
data['location'] = data['location'].map(lambda x: x.split(','))
data['location'] = data['location'].map(lambda x: x[1] if len(x)==2 else x[0])

In [6]:
#split salary to min and max
data['salary'] = data['salary'].map(lambda x: x.split())
data['min_salary'] = data['salary'].map(lambda x: x[1] if len(x)>2 else '')
data['max_salary'] = data['salary'].map(lambda x: x[-1] if len(x)>2 else '')

In [7]:
#converting min and max salary to int
data['min_salary'] = data['min_salary'].map(lambda x: x.replace('.',''))
data['max_salary'] = data['max_salary'].map(lambda x: x.replace('.',''))
data['min_salary'] = data['min_salary'].map(lambda x: int(x) if '0' in x else 0)
data['max_salary'] = data['max_salary'].map(lambda x: int(x) if '0' in x else 0)

In [9]:
#filter irrelevant salary
#there is salary data with USD currency
data = data[(data['min_salary']>100000) | (data['min_salary']==0)]

In [10]:
data.shape

(449, 8)

In [11]:
#convert job_title to lowercase, for easy regex
data['job_title'] = data['job_title'].map(lambda x: x.lower())

In [12]:
data.head()

,job_title,job_company,job_details,location,experience,salary,min_salary,max_salary
0,it data engineer / data analytics,PT Kawan Lama Sejahtera,"[Jakarta, IDR 10.000.000 - 20.000.000, 3 – 5 t...",Jakarta,3 – 5 tahun,"[IDR, 10.000.000, -, 20.000.000]",10000000,20000000
1,data engineer,Mapan - PT Ruma (Rekan Usaha Mikro Anda),"[SCBD, Jakarta, 1 – 3 tahun]",Jakarta,1 – 3 tahun,[],0,0
2,business analyst (afees),AIA Singapore Pte Ltd,"[Batam, 3 – 5 tahun]",Batam,3 – 5 tahun,[],0,0
3,e-commerce specialist (remote),Etaily,"[Jakarta, Login to view estimated salary by Gl...",Jakarta,3 – 5 tahun,"[Login, to, view, estimated, salary, by, Glints.]",0,0
4,digital marketing manager (remote),Hepelle,"[Jakarta, Login to view estimated salary by Gl...",Jakarta,3 – 5 tahun,"[Login, to, view, estimated, salary, by, Glints.]",0,0


In [13]:
#specifi experience to minimun experience and conert to int
data['min_experience'] = data.experience.map(lambda x: x.split())
data['min_experience'] = data['min_experience'].map(lambda x: x[0] if len(x)> 2 else '0')
data['min_experience'] = data['min_experience'].replace('Kurang', '0')
data['min_experience'] = data['min_experience'].map(lambda x: int(x))

In [14]:
data.shape

(449, 9)

In [15]:
data.min_experience.unique()

array([3, 1, 0, 5], dtype=int64)

In [16]:
data.head()

,job_title,job_company,job_details,location,experience,salary,min_salary,max_salary,min_experience
0,it data engineer / data analytics,PT Kawan Lama Sejahtera,"[Jakarta, IDR 10.000.000 - 20.000.000, 3 – 5 t...",Jakarta,3 – 5 tahun,"[IDR, 10.000.000, -, 20.000.000]",10000000,20000000,3
1,data engineer,Mapan - PT Ruma (Rekan Usaha Mikro Anda),"[SCBD, Jakarta, 1 – 3 tahun]",Jakarta,1 – 3 tahun,[],0,0,1
2,business analyst (afees),AIA Singapore Pte Ltd,"[Batam, 3 – 5 tahun]",Batam,3 – 5 tahun,[],0,0,3
3,e-commerce specialist (remote),Etaily,"[Jakarta, Login to view estimated salary by Gl...",Jakarta,3 – 5 tahun,"[Login, to, view, estimated, salary, by, Glints.]",0,0,3
4,digital marketing manager (remote),Hepelle,"[Jakarta, Login to view estimated salary by Gl...",Jakarta,3 – 5 tahun,"[Login, to, view, estimated, salary, by, Glints.]",0,0,3


In [17]:
#replacing job title with specify title, to easy filtering
data.loc[data['job_title'].str.contains('data analytic'), 'job_title'] = 'data analytic'
data.loc[data['job_title'].str.contains('data analyst'), 'job_title'] = 'data analyst'
data.loc[data['job_title'].str.contains('data scientist'), 'job_title'] = 'data scientist'
data.loc[data['job_title'].str.contains('data engineer'), 'job_title'] = 'data engineer'
data.loc[data['job_title'].str.contains('business analyst'), 'job_title'] = 'business analyst'
data.loc[data['job_title'].str.contains('system analyst'), 'job_title'] = 'system analyst '

In [19]:
data.head()

,job_title,job_company,job_details,location,experience,salary,min_salary,max_salary,min_experience
0,data analytic,PT Kawan Lama Sejahtera,"[Jakarta, IDR 10.000.000 - 20.000.000, 3 – 5 t...",Jakarta,3 – 5 tahun,"[IDR, 10.000.000, -, 20.000.000]",10000000,20000000,3
1,data engineer,Mapan - PT Ruma (Rekan Usaha Mikro Anda),"[SCBD, Jakarta, 1 – 3 tahun]",Jakarta,1 – 3 tahun,[],0,0,1
2,business analyst,AIA Singapore Pte Ltd,"[Batam, 3 – 5 tahun]",Batam,3 – 5 tahun,[],0,0,3
3,e-commerce specialist (remote),Etaily,"[Jakarta, Login to view estimated salary by Gl...",Jakarta,3 – 5 tahun,"[Login, to, view, estimated, salary, by, Glints.]",0,0,3
4,digital marketing manager (remote),Hepelle,"[Jakarta, Login to view estimated salary by Gl...",Jakarta,3 – 5 tahun,"[Login, to, view, estimated, salary, by, Glints.]",0,0,3


In [20]:
#checking location data
data.location.unique()

array(['Jakarta', ' Jakarta', 'Batam', 'Tangerang', 'Surabaya',
       'Yogyakarta', 'Semarang', 'Mataram', 'Bandung', 'Depok', 'Senayan',
       'Abang', 'Tasikmalaya', 'Denpasar', 'Sleman', 'Serang', 'Bogor',
       'Bandar Lampung', 'South Tangerang', 'Kemang', 'Malang', 'Solo',
       'Balikpapan', 'Serpong', 'Bekasi'], dtype=object)

In [21]:
data['location'] = data.location.map(lambda x: x.strip())

In [22]:
#here cleaning some data with unsatisfied value
data.loc[data['location']=='South Tangerang','location'] = 'Tangerang'
data.loc[data['location']=='Kemang','location'] = 'Jakarta'
data.loc[data['location']=='Abang','location'] = 'Jakarta'
data.loc[data['location']=='Serpong','location'] = 'Jakarta'
data.loc[data['location']=='Senayan','location'] = 'Jakarta'

In [24]:
data.head()

,job_title,job_company,job_details,location,experience,salary,min_salary,max_salary,min_experience
0,data analytic,PT Kawan Lama Sejahtera,"[Jakarta, IDR 10.000.000 - 20.000.000, 3 – 5 t...",Jakarta,3 – 5 tahun,"[IDR, 10.000.000, -, 20.000.000]",10000000,20000000,3
1,data engineer,Mapan - PT Ruma (Rekan Usaha Mikro Anda),"[SCBD, Jakarta, 1 – 3 tahun]",Jakarta,1 – 3 tahun,[],0,0,1
2,business analyst,AIA Singapore Pte Ltd,"[Batam, 3 – 5 tahun]",Batam,3 – 5 tahun,[],0,0,3
3,e-commerce specialist (remote),Etaily,"[Jakarta, Login to view estimated salary by Gl...",Jakarta,3 – 5 tahun,"[Login, to, view, estimated, salary, by, Glints.]",0,0,3
4,digital marketing manager (remote),Hepelle,"[Jakarta, Login to view estimated salary by Gl...",Jakarta,3 – 5 tahun,"[Login, to, view, estimated, salary, by, Glints.]",0,0,3


In [26]:
#saving data to csv, to process in Tableau
data.to_csv("data_webscrape.csv")